In [ ]:
import pickle
import re
import time
import warnings

import matplotlib.pyplot as plt
import nltk
import pandas as pd
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    accuracy_score,
    classification_report,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

nltk.download("omw-1.4")
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [ ]:
df = pd.read_csv("./labeled_data/labeled_foi_text_2023_05_01.csv")

In [ ]:
ps = PorterStemmer()  # creating an instance of the class

In [ ]:
def preprocess_text(text):
    """In preprocess_text function we will apply all the things that are given below:
    - removing special characters
    - removing punctuations
    - removing numbers
    - removing stopwords
    - doing stemming
    - transforming in lowercase
    - removing excessive whitespaces
    """
    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", str(text))

    # remove punctuations in string
    text = re.sub(r"[^\w\s]", "", str(text))

    # 1. tokenize
    tokens = nltk.word_tokenize(text)

    # 2. check if stopword and stemming the word
    tokens = [ps.stem(w) for w in tokens if w.lower() not in stopwords.words("english")]

    # 3. join back together
    text = " ".join(tokens)

    # return text in lower case and stripped of whitespaces
    text = text.lower().strip()

    return text

In [ ]:
df["clean_text"] = df["FOI_TEXT"].apply(lambda x: preprocess_text(x))
df.head()

In [ ]:
# define a function for getting all words from the text


def returning_tokinize_list(df, column_name):
    df = df.reset_index(drop=True)
    tokenize_list = [nltk.word_tokenize(df[column_name][i]) for i in range(df.shape[0])]
    final = [j for i in tokenize_list for j in i]
    return final

In [ ]:
# get the all words of text into list
tokenize_list_words = returning_tokinize_list(df, "clean_text")

In [ ]:
# function for words in dataframe format


def table_format(data_list, column_name):
    df_ = pd.DataFrame(data_list, columns=[column_name, "Frequency_distribution"])
    return df_


# function for extracting the most common words in reviews text
def most_common_words(cleaned_col_name_list, common_words=10):
    fdist = FreqDist(cleaned_col_name_list)
    most_common = fdist.most_common(common_words)
    return most_common

In [ ]:
label_enc = LabelEncoder()
df["PRIMARY_LABEL_ENC"] = label_enc.fit_transform(df["PRIMARY_LABEL"])

In [ ]:
label_enc.classes_

In [ ]:
X = df["clean_text"]  # independent feature
y = df["PRIMARY_LABEL_ENC"]  # dependent feature

In [ ]:
def splitting_dataset(X, y, test_size=0.25):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=0, stratify=y
    )
    return X_train, X_test, y_train, y_test

In [ ]:
def get_features_from_text(X):
    # Fit and transform the data to a document-term matrix using TfidfVectorizer
    tfidf = TfidfVectorizer(min_df=2)
    X_tfidff = tfidf.fit_transform(X.values)
    print("Number of features TF-IDF: %d" % len(tfidf.get_feature_names_out()))

    with open("./labeled_data/xgb_tfidf.pickle", "wb") as f:
        pickle.dump(tfidf, f)

    return X_tfidff.toarray()

In [ ]:
X_tfidff = get_features_from_text(X)

In [ ]:
oversample = SMOTE(random_state=42)
overX, overY = oversample.fit_resample(X_tfidff, y)
pd.DataFrame(overY)["PRIMARY_LABEL_ENC"].value_counts()

In [ ]:
# split data into 75% training and 25% testing set
X_train_tfidf, X_test_tfidf, y_train, y_test = splitting_dataset(
    overX, overY, test_size=0.25
)

In [ ]:
def results(model, y_pred, y_test):
    accuracy = round(accuracy_score(y_pred, y_test) * 100, 4)
    precision = round(precision_score(y_pred, y_test, average="macro") * 100, 4)
    recall = round(recall_score(y_pred, y_test, average="macro") * 100, 4)
    f1_scr = round(f1_score(y_pred, y_test, average="macro") * 100, 4)

    print("\nAccuracy: {}%".format(accuracy))
    print("Precision: {}%".format(precision))
    print("Recall: {}%".format(recall))
    print("F1-Score: {}%".format(f1_scr))
    print()
    print("Classification Report:")
    print(classification_report(y_pred, y_test, target_names=label_enc.classes_))
    print()
    print("Confusion Matrix:")
    fig, ax = plt.subplots(figsize=(5, 5))
    ConfusionMatrixDisplay.from_predictions(
        y_pred,
        y_test,
        ax=ax,
        display_labels=label_enc.classes_,
        xticks_rotation="vertical",
    )
    plt.show()

    return {
        "Model": model.__class__.__name__,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1": f1_scr,
    }

In [ ]:
def train_model(model):
    # Record the start time
    start_time = time.time()

    # Fit the model on the training data
    model.fit(X_train_tfidf, y_train)

    # Record the end time
    end_time = time.time()

    # Compute the training time in seconds
    training_time = end_time - start_time
    print(f"Training time: {training_time:.3f} seconds")

    # predict the model on testing data
    y_pred = model.predict(X_test_tfidf)

    # get results
    RESULTS_LR = results(model, y_pred, y_test)

    # Add the training time to RESULTS_LR
    RESULTS_LR["Training time (s)"] = training_time

    return RESULTS_LR

In [ ]:
def get_model_results(res):
    # Convert the dictionary to a DataFrame
    return pd.DataFrame.from_dict([res]).set_index("Model")

In [ ]:
model_tuple = [
    (
        "XGBoost",
        xgb.XGBClassifier(
            **{
                "learning_rate": 0.1,
                "max_depth": 5,
                "n_estimators": 100,
                "num_class": 5,
                "objective": "multi:softmax",
                "random_state": 42,
            }
        ),
    )
]

In [ ]:
for model_name, model in tqdm(model_tuple, desc="model running"):
    print("=" * 100)
    print(f"\t\t\t {model_name}")
    print("=" * 100)

    # Record the start time
    start_time = time.time()

    # Fit the model on the training data
    model.fit(X_train_tfidf, y_train)

    # Record the end time
    end_time = time.time()

    # Compute the training time in seconds
    training_time = end_time - start_time
    print(f"Training time: {training_time:.3f} seconds")

    # predict the model on testing data
    y_pred = model.predict(X_test_tfidf)

    # get results
    RESULT = results(model, y_pred, y_test)

    # Add the training time to RESULTS_LR
    RESULT["Training time (s)"] = training_time

    df_results = get_model_results(RESULT)
    print(df_results)
    print()

In [ ]:
# Save the trained model to a file
model.save_model("./labeled_data/xgb_model.xgb")